<a href="https://colab.research.google.com/github/nahuel89p/whisperx_diarization/blob/main/Transcription_with_diarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word by word transcriptions with sepaker diarization 🎙️✨

---

In [ ]:
!pip install git+https://github.com/m-bain/whisperx.git --upgrade

  Cloning https://github.com/m-bain/whisperx.git to /tmp/pip-req-build-la13z93d
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperx.git /tmp/pip-req-build-la13z93d
  Resolved https://github.com/m-bain/whisperx.git to commit b1a98b78c9152ace9f9801593b5fa0c7d5d96b0f
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/pyannote/pyannote-audio (to revision 11b56a137a578db9335efc00298f6ec1932e6317) to /tmp/pip-install-13443elk/pyannote-audio_68b9046845c343d4b15fea1ce88751b9
  Running command git clone --filter=blob:none --quiet https://github.com/pyannote/pyannote-audio /tmp/pip-install-13443elk/pyannote-audio_68b9046845c343d4b15fea1ce88751b9
  Running command git rev-parse -q --verify 'sha^11b56a137a578db9335efc00298f6ec1932e6317'
  Running command git fetch -q https://github.com/pyannote/pyannote-audio 11b56a137a578db9335efc00298f6ec1932e6317
  Running command git checkout -q 11b56a137a578db9335efc00298f6ec1932e6317
  Resolved https:/

## Upload mp4 files 🎵


In [ ]:
from google.colab import files

uploaded = files.upload()
audio_file = list(uploaded.keys())[0]

Saving QV-69w-51zc.mp4 to QV-69w-51zc.mp4
Saving VQHPDtX358Y.mp4 to VQHPDtX358Y.mp4
Saving MemAKOLtiGM.mp4 to MemAKOLtiGM.mp4
Saving LtUD98tkLPw.mp4 to LtUD98tkLPw.mp4
Saving epIFO7OheTI.mp4 to epIFO7OheTI.mp4
Saving xDZOQMYRw6w.mp4 to xDZOQMYRw6w.mp4
Saving Tr6kX-1wvgM.mp4 to Tr6kX-1wvgM.mp4
Saving buS-n3wh6vg.mp4 to buS-n3wh6vg.mp4
Saving x1R-VTMknTo.mp4 to x1R-VTMknTo.mp4


## Transcription 🗣️


In [ ]:
import whisperx
import gc
from google.colab import files

filenames=list(uploaded.keys())
results = []
device = "cuda"
batch_size = 16
compute_type = "float16"
model = whisperx.load_model("large-v2", device, compute_type=compute_type)


No language specified, language will be first be detected for each audio file (increases inference time).


100%|██████████████████████████████████████| 16.9M/16.9M [00:00<00:00, 116MiB/s]
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.0.9. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.1+cu118. Bad things might happen unless you revert torch to 1.x.


In [ ]:
hf_token = ''
for audio_file in filenames:
  # Transcript
  audio = whisperx.load_audio(audio_file)
  result = model.transcribe(audio, batch_size=batch_size)
  print(result["segments"])

  # Align
  model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
  result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

  diarize_model = whisperx.DiarizationPipeline(use_auth_token=hftoken, device=device)

  # add min/max number of speakers if known
  diarize_segments = diarize_model(audio) #auto
  # diarize_model(audio, min_speakers=min_speakers, max_speakers=max_speakers)

  result = whisperx.assign_word_speakers(diarize_segments, result)
  import json
  file_name= str(audio_file.split('.')[0])+'.json'
  # Saving the entire result as a JSON file
  with open(file_name, 'w') as file:
      json.dump(result, file, indent=4)

  # Download the JSON file
  files.download(file_name)
  results.append(result)


Detected language: en (1.00) in first 30s of audio...
[{'text': " Welcome to Palisades Gold Radio. I'm your host, Tom Bodrovics. Joining me today is Patrick Karim from NorthStarBadShorts.com. How are you today, Pat? I'm doing great. Always fun catching up with you there, especially our conversations even before we start the show, right? I'm telling you, one day, as soon as we hook up, I said, hold on, Patrick, we should record this, show people.", 'start': 1.477, 'end': 23.397}, {'text': " all the discussions before, because it's just, I don't know, it's just like a fun conversation there. I feel like, you know, if we make me meet up one day, it's just going to be like, hey, Tom, let's go grab a beer. Let's go have fun. And look, I guess Zoom is the next best thing we have there, but I'll go to your neck of the woods for sure one day there. It sure makes that whatever, seven or what would it be like 5,000 mile commute a little bit easier, but", 'start': 23.566, 'end': 50.633}, {'text':

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.0.9. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../root/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.1+cu118. Bad things might happen unless you revert torch to 1.x.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Detected language: en (0.99) in first 30s of audio...
[{'text': " Happy to welcome to Forward Guidance, Andrew Beer, managing member of Dynamic Beta Investments. Andrew, good to have you on Forward Guidance. Dynamic Beta, why beta? Isn't everyone trying to pursue alpha in this world? Why are you focusing on beta?", 'start': 4.733, 'end': 20.444}, {'text': " It's a good question. So actually, we just changed our name to DBI officially as of last week. I think people often talk about alpha and beta being separate. Beta is the cheap, easy thing that anybody can get, whereas alpha is this esoteric thing. The reality is they're interconnected. If you buy equities at the right time or you buy bonds at the right time, you can generate an enormous amount of alpha through those kinds of", 'start': 20.444, 'end': 46.718}, {'text': " regular decisions over time. So for us, dynamic beta, what it really meant was that if you want to understand how hedge funds are making money, um, then the thing yo

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.0.9. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../root/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.1+cu118. Bad things might happen unless you revert torch to 1.x.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>